In [1]:
import torch
from robofab.place import poses_to_frames
from curobo.types.robot import RobotConfig
from curobo.geom.types import Pose
from curobo.wrap.reacher.ik_solver import IKSolver, IKSolverConfig
from curobo.geom.types import WorldConfig, Mesh, Cylinder, Cuboid
from curobo.geom.types import Pose
import trimesh
from curobo.types.base import TensorDeviceType
from curobo.types.state import JointState
from curobo.wrap.reacher.motion_gen import MotionGen, MotionGenConfig, MotionGenPlanConfig
from curobo.util.trajectory import InterpolateType
import numpy as np
from curobo.geom.transform import matrix_to_quaternion


tensor_args = TensorDeviceType()


def get_kuka_ik_solver(gripper_width=0.15, world_config: WorldConfig = None, num_seeds=500):

    config_file, _ = load_kuka_kin_model(gripper_width)
    robot_cfg = RobotConfig.from_dict(config_file)
    ik_config = IKSolverConfig.load_from_robot_config(
        robot_cfg,
        world_model=world_config,
        tensor_args=tensor_args,
        use_cuda_graph=True,
        rotation_threshold=0.001,
        position_threshold=0.001,
        num_seeds=num_seeds,
        self_collision_check=False,
        self_collision_opt=False,
        collision_activation_distance=0.02
    )
    ik_solver = IKSolver(ik_config)
    return ik_solver


### initialize Plan IK function
def get_poses_by_flag(poses: Pose, flag: torch.tensor):
    return Pose(position=poses.position[flag, :], quaternion=poses.quaternion[flag, :])


def plan_ik(poses: Pose, world_config: WorldConfig, gripper_width: float = 0.15):
    ik_solver = get_kuka_ik_solver(gripper_width=gripper_width, world_config=world_config)
    result = ik_solver.solve_batch(poses)
    ik_solver.reset_cuda_graph()
    if result.success.any():
        flag = result.success
        success_qs = result.solution[result.success]
        sucess_pose = get_poses_by_flag(poses, flag.flatten())
        return (success_qs, sucess_pose, flag.flatten())
    else:
        return None


def plan_ik_seed_config(poses: Pose, world_config: WorldConfig, seed_config: JointState,
                        gripper_width: float = 0.15):
    ik_solver = get_kuka_ik_solver(gripper_width=gripper_width, world_config=world_config, num_seeds=1)
    seed_config = seed_config.position.view(1, -1)
    result = ik_solver.solve_batch(goal_pose=poses, seed_config=seed_config, num_seeds=1)
    if result.success.any():
        flag = result.success
        success_qs = result.solution[result.success]
        sucess_pose = get_poses_by_flag(poses, flag.flatten())
        return (success_qs, sucess_pose, flag.flatten())
    else:
        return None



def get_kuka_motion_gen(gripper_width=0.15, world_config: WorldConfig = None, num_seeds=1000):
    # config_file, _ = load_franka_kin_model(gripper_width)
    config_file, _ = load_kuka_kin_model(gripper_width)
    robot_cfg = RobotConfig.from_dict(config_file)
    motion_gen_config = MotionGenConfig.load_from_robot_config(
        robot_cfg,
        world_model=world_config,
        tensor_args=tensor_args,
        use_cuda_graph=True,
        rotation_threshold=0.05,
        position_threshold=0.005,
        num_ik_seeds=num_seeds,
        num_trajopt_seeds=num_seeds,
        interpolation_dt=0.01,
        interpolation_steps=1000,
        interpolation_type=InterpolateType.CUBIC,
        high_precision=False,
        self_collision_check=False,
        self_collision_opt=False,
        minimize_jerk=False,
        collision_activation_distance=0.02)
    return MotionGen(motion_gen_config)



def plan_motion_attached_object(start_state: JointState,
                                goal_poses: Pose,
                                attached_object: Mesh,
                                attached_state: JointState,
                                world_config: WorldConfig,
                                gripper_width: float = 0.15):
    motion_gen = get_kuka_motion_gen(world_config=world_config, gripper_width=gripper_width)
    place_poses = Pose(position=goal_poses.position.view(1, -1, 3), quaternion=goal_poses.quaternion.view(1, -1, 4))
    start_state.joint_names = motion_gen.kinematics.joint_names.copy()

    attached_state.joint_names = motion_gen.kinematics.joint_names.copy()
    motion_gen.attach_external_objects_to_robot(attached_state,
                                                [attached_object],
                                                surface_sphere_radius=0.015,
                                                link_name="attached_object")

    result = motion_gen.plan_goalset(start_state, place_poses, MotionGenPlanConfig(max_attempts=100))
    if result.success.any():
        qtraj = result.get_interpolated_plan().position
        return (qtraj, result.goalset_index.item())
    else:
        return None


def plan_motion_js_attached_object(start_state: JointState,
                                   goal_state: JointState,
                                   attached_object: Mesh,
                                   attached_state: JointState,
                                   world_config: WorldConfig,
                                   gripper_width: float = 0.15):
    motion_gen = get_kuka_motion_gen(world_config=world_config, gripper_width=gripper_width)
    start_state.joint_names = motion_gen.kinematics.joint_names.copy()
    goal_state.joint_names = motion_gen.kinematics.joint_names.copy()

    attached_state.joint_names = motion_gen.kinematics.joint_names.copy()
    motion_gen.attach_external_objects_to_robot(attached_state,
                                                [attached_object],
                                                surface_sphere_radius=0.015,
                                                link_name="attached_object")

    result = motion_gen.plan_single_js(start_state, goal_state, MotionGenPlanConfig(max_attempts=100))
    if result.success.any():
        qtraj = result.get_interpolated_plan().position
        return qtraj
    else:
        return None


def compute_pick_place(tube_id, tube_pose, tube_mesh, place_world, robot_kin_model):
    ## 2.1 Compute place poses
    place_result = plan_ik(tube_pose, place_world, 0.15)
    if place_result is None:
        raise RuntimeError("No solution found!")

    # update
    place_poses = place_result[1]

    ## 2.3 Plan Place IK
    ### Plan place approach IK
    place_approach_poses = place_poses.clone()
    place_approach_poses.position += torch.tensor([0, 0, 0.1], **tensor_args.as_torch_dict())
    place_approach_result = plan_ik(place_approach_poses, place_world, 0.15)
    if place_approach_result is None:
        raise RuntimeError("No solution found!")

    # update
    place_approach_poses = place_approach_result[1]
    place_poses = get_poses_by_flag(place_poses, place_approach_result[2])
    # print("Number of place poses", place_poses.position.shape[0])

    ### Compute place to pick transformation matrix
    pick_pose_pos = [-1.5, -1.5, 0]
    pick_pose_quat = [0, 1, -1, 0]
    pick_poses = Pose(position=torch.tensor([pick_pose_pos] * len(place_poses), **tensor_args.as_torch_dict()),
                      quaternion=torch.tensor([pick_pose_quat] * len(place_poses), **tensor_args.as_torch_dict()))
    pick_frames = poses_to_frames(pick_poses)

    ### Compute pick poses
    from curobo.geom.transform import matrix_to_quaternion
    pick_frames = torch.tensor(pick_frames, **tensor_args.as_torch_dict())
    pick_poses = Pose(position=pick_frames[:, :3, 3], quaternion=matrix_to_quaternion(pick_frames[:, :3, :3]))

    ### Plan pick ik
    pick_result = plan_ik(pick_poses, place_world, 0.15)
    if pick_result is None:
        raise RuntimeError("No solution found!")

    # update
    pick_poses = pick_result[1]
    place_poses = get_poses_by_flag(place_poses, pick_result[2])
    place_approach_poses = get_poses_by_flag(place_approach_poses, pick_result[2])

    ### Plan pick approach ik
    pick_approach_poses = pick_poses.clone()
    pick_approach_poses.position += torch.tensor([0, 0, 0.1], **tensor_args.as_torch_dict())
    pick_approach_result = plan_ik(pick_approach_poses, place_world, 0.15)
    if pick_approach_result is None:
        raise RuntimeError("No solution found!")

    # update
    pick_approach_poses = pick_approach_result[1]
    pick_poses = get_poses_by_flag(pick_poses, pick_approach_result[2])
    place_poses = get_poses_by_flag(place_poses, pick_approach_result[2])
    # print("Number of pick poses", place_poses.position.shape[0])

    ## 4.1 Retract -> pick_approach
    ### Choose one pick-place-approach pair

    pick_place_id = 0
    pick_pose = pick_poses.clone()[pick_place_id]
    place_pose = place_poses.clone()[pick_place_id]
    pick_approach_pose = pick_approach_poses.clone()[pick_place_id]
    place_approach_pose = place_approach_poses.clone()[pick_place_id]

    ### Compute attach objects and states

    mesh = trimesh.creation.cylinder(radius=0.024, height=0.75, sections=16)
    mesh.apply_translation(pick_pose_pos)
    mesh = Mesh(
        name=f"tube_mesh{tube_id}",
        pose=[*pick_pose_pos, *pick_pose_quat],
        vertices=mesh.vertices,
        faces=mesh.faces,
        color=[0.7, 0.7, 0.7, 1.0],
    )

    # pick_result = plan_ik(pick_pose, pick_world[3], 0.15)
    pick_result = plan_ik(pick_pose, place_world, 0.15)
    if pick_result is None:
        raise RuntimeError("No solution found!")
    pick_q = pick_result[0].cpu().numpy().reshape(1, -1)

    attached_state = JointState.from_position(pick_result[0])
    attached_object = mesh
    start_state = JointState.from_position(robot_kin_model.retract_config.view(1, -1),
                                           joint_names=robot_kin_model.joint_names)

    attached_object_for_rendering = trimesh.Trimesh(attached_object.vertices.copy(), attached_object.faces.copy())
    T = poses_to_frames(pick_pose).reshape(4, 4)
    invT = np.linalg.inv(T)
    attached_object_for_rendering.apply_transform(invT)

    ### Plan motion
    trajs_retract_pick_approach_result = plan_motion_attached_object(start_state, pick_approach_pose, attached_object,
                                                                     attached_state, place_world, 0.15)
    if trajs_retract_pick_approach_result is None:
        raise RuntimeError("No solution found!")
    # trajs_retract_pick = trajs_retract_pick_approach_result[0].clone().cpu().numpy()
    # trajs_retract_pick = np.vstack([trajs_retract_pick, pick_q])

    ### Resolve IK Jumping
    pick_approach_state = JointState.from_position(trajs_retract_pick_approach_result[0][-1],
                                                   joint_names=robot_kin_model.joint_names)
    pick_result = plan_ik_seed_config(pick_pose, place_world, pick_approach_state, 0.15)
    if pick_result is None:
        raise RuntimeError("No solution found!")
    pick_q = pick_result[0].cpu().numpy()
    trajs_retract_pick = trajs_retract_pick_approach_result[0].clone().cpu().numpy()
    trajs_retract_pick = np.vstack([trajs_retract_pick, pick_q])

    ## 4.2 retract -> place_approach
    ### Compute attach objects and states

    place_result = plan_ik(place_pose, place_world, 0.15)
    if place_result is None:
        raise RuntimeError("No solution found!")
    # place_q = place_result[0].cpu().numpy().reshape(1, -1)
    attached_state = JointState.from_position(place_result[0])
    attached_object = tube_mesh
    start_state = JointState.from_position(robot_kin_model.retract_config.view(1, -1),
                                           joint_names=robot_kin_model.joint_names)

    attached_object_for_rendering = trimesh.Trimesh(attached_object.vertices.copy(), attached_object.faces.copy())
    T = poses_to_frames(place_pose).reshape(4, 4)
    invT = np.linalg.inv(T)
    attached_object_for_rendering.apply_transform(invT)

    ### Plan motion
    trajs_retract_place_approach_result = plan_motion_attached_object(start_state, place_approach_pose, attached_object,
                                                                      attached_state, place_world, 0.02)
    if trajs_retract_place_approach_result is None:
        raise RuntimeError("No solution found!")
    # trajs_retract_place = trajs_retract_place_approach_result[0].cpu().numpy()
    # trajs_retract_place = np.vstack([trajs_retract_place, place_q])

    ### Resolve IK Jumping
    place_approach_state = JointState.from_position(trajs_retract_place_approach_result[0][-1],
                                                    joint_names=robot_kin_model.joint_names)
    place_result = plan_ik_seed_config(place_pose, place_world, place_approach_state, 0.15)
    if place_result is None:
        raise RuntimeError("No solution found!")
    place_q = place_result[0].cpu().numpy()
    trajs_retract_place = trajs_retract_place_approach_result[0].clone().cpu().numpy()
    trajs_retract_place = np.vstack([trajs_retract_place, place_q])

    ## 4.3 pick_approach -> place_approach
    pick_approach_q = torch.tensor(trajs_retract_pick[-2, :].reshape(1, -1), **tensor_args.as_torch_dict())
    place_approach_q = torch.tensor(trajs_retract_place[-2, :].reshape(1, -1), **tensor_args.as_torch_dict())
    start_state = JointState.from_position(pick_approach_q, joint_names=robot_kin_model.joint_names)
    goal_state = JointState.from_position(place_approach_q, joint_names=robot_kin_model.joint_names)

    trajs_pick_place_approach_result = plan_motion_js_attached_object(start_state, goal_state, attached_object,
                                                                      attached_state, place_world, 0.15)
    if trajs_pick_place_approach_result is None:
        raise RuntimeError("No solution found!")
    trajs_pick_place = trajs_pick_place_approach_result.cpu().numpy()
    trajs_pick_place = np.vstack([pick_q, trajs_pick_place, place_q, trajs_pick_place[::-1], pick_q])

    return trajs_pick_place, attached_object_for_rendering

In [2]:
# torch
import torch
import numpy as np
from curobo.cuda_robot_model.cuda_robot_model import CudaRobotModel, CudaRobotModelConfig
from curobo.types.base import TensorDeviceType
from curobo.types.robot import RobotConfig
from curobo.util_file import get_robot_path, join_path, load_yaml
from curobo.geom.types import WorldConfig, Mesh, Cylinder, Cuboid
from robofab import ROBOFAB_DATA_DIR
import numpy as np
from scipy.spatial.transform import Rotation as R
from curobo.geom.types import Pose


tensor_args = TensorDeviceType()


def load_kuka_kin_model(gripper_width = 0.15):
    robot_name = "robot"
    config_file = load_yaml(ROBOFAB_DATA_DIR+ f"/robot/kuka/cfg.yaml")["robot_cfg"]
    config_kinematics = config_file['kinematics']
    config_kinematics["urdf_path"] = ROBOFAB_DATA_DIR + "/robot/kuka/" + config_kinematics["urdf_path"]
    config_kinematics["collision_spheres"] = ROBOFAB_DATA_DIR + "/robot/kuka/" + config_kinematics["collision_spheres"]
    config_kinematics["lock_joints"] = {"left_gripper_finger_joint": gripper_width, "right_gripper_finger_joint": gripper_width}

    robot_cfg = RobotConfig.from_dict(config_file, tensor_args)
    kin_model = CudaRobotModel(robot_cfg.kinematics)
    return config_file, kin_model


def rotate_euler(euler_angles, axis, angle, degrees=False):
    """
    Rotate given Euler angles by a specified angle around an arbitrary axis.

    Parameters:
    - euler_angles: tuple or list of 3 floats, Euler angles (roll, pitch, yaw) in radians (or degrees if degrees=True).
    - axis: tuple or list of 3 floats, the arbitrary axis vector (will be normalized).
    - angle: float, the rotation angle in radians (or degrees if degrees=True).
    - degrees: bool, if True, input angles are in degrees; otherwise, in radians.

    Returns:
    - new_euler: tuple of 3 floats, the new Euler angles after rotation, in the same units as input.
    """
    # Normalize the axis vector
    axis = np.array(axis)
    axis /= np.linalg.norm(axis)

    # Create rotation from Euler angles
    rot_euler = R.from_euler('xyz', euler_angles, degrees=degrees)

    # Create rotation from axis-angle
    rot_axis_angle = R.from_rotvec(axis * angle if not degrees else axis * np.deg2rad(angle))

    # Compose the rotations: apply the new rotation to the existing one
    # Note: Order matters; here we apply the new rotation after the original (right-multiply)
    # If you want to apply it before, use rot_axis_angle * rot_euler
    new_rot = rot_euler * rot_axis_angle

    # Convert back to Euler angles
    new_euler = new_rot.as_euler('xyz', degrees=degrees)

    return tuple(new_euler)


def euler_to_quaternion(roll, pitch, yaw):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return [qx, qy, qz, qw]


def gen_fully_placed_world(scaffold_structure, valid_tube_index, world_cfg):
    tube_cylinder = []
    tube_mesh = []
    for tube_id, tube in scaffold_structure.items():
        if tube_id not in valid_tube_index:
            continue

        start_p = tube["from_p"]
        start_p[0] -= 0.2
        end_p = tube["to_p"]
        end_p[0] -= 0.2

        # get rotation quaternion from start_p to end_p
        direction = np.array(end_p) - np.array(start_p)
        direction = direction / np.linalg.norm(direction)
        # Assuming the cylinder's default orientation is along the z-axis
        z_axis = np.array([0, 0, 1])
        # Compute euler angles
        if np.allclose(direction, z_axis):
            roll, pitch, yaw = 0, 0, 0
        elif np.allclose(direction, -z_axis):
            roll, pitch, yaw = 0, np.pi, 0
        else:
            roll = np.arctan2(direction[0], direction[1])
            yaw = np.arccos(direction[2])
            pitch = 0
        qx, qy, qz, qw = euler_to_quaternion(roll, pitch, yaw)

        cylinder = Cylinder(
            name=f"tube{tube_id}",
            radius=0.024,
            height=np.linalg.norm(np.array(end_p) - np.array(start_p)),
            # pose is [x, y, z, qx, qy, qz, qw]
            pose=[(start_p[0] + end_p[0]) / 2, (start_p[1] + end_p[1]) / 2, (start_p[2] + end_p[2]) / 2, qx, qy, qz,
                  qw],
            color=[0.7, 0.7, 0.7, 1.0],
        )
        tube_cylinder.append(cylinder)
        mesh = WorldConfig(cylinder=[cylinder]).get_mesh_world(merge_meshes=True).mesh[0]  # .get_trimesh_mesh()
        tube_mesh.append(mesh)

    fully_placed_world = WorldConfig(
        cylinder=[*world_cfg.cylinder] + tube_cylinder,
        cuboid=[*world_cfg.cuboid],
    )
    return fully_placed_world, tube_mesh, tube_cylinder


def update_frame(original_quat, new_pos, start_point, end_point):
    """
    Updates the frame to a new position and rotates it so that its z-axis aligns with the direction from start_point to end_point.

    Parameters:
    - original_pos: np.array (3,) - Original position (not used for new position, but included for completeness).
    - original_quat: np.array (4,) - Original quaternion in [x, y, z, w] format.
    - new_pos: np.array (3,) - New position for the frame.
    - start_point: np.array (3,) - Starting point for the z-axis direction.
    - end_point: np.array (3,) - Ending point for the z-axis direction.

    Returns:
    - new_pos: np.array (3,) - The new position.
    - new_quat: np.array (4,) - The updated quaternion in [x, y, z, w] format.
    """
    # Compute the target z-direction
    direction = end_point - start_point
    if np.linalg.norm(direction) == 0:
        raise ValueError("Start and end points cannot be the same; direction vector would be zero.")
    new_z = direction / np.linalg.norm(direction)

    # Get the current rotation and z-axis
    rot = R.from_quat(original_quat)
    current_z = rot.apply(np.array([0, 0, 1]))

    # If already aligned, no rotation needed
    if np.allclose(current_z, new_z, atol=1e-6):
        new_rot = rot
    else:
        # Compute the axis and angle for the alignment rotation
        cross_prod = np.cross(current_z, new_z)
        axis_norm = np.linalg.norm(cross_prod)
        if axis_norm == 0:
            # If vectors are opposite, choose an arbitrary perpendicular axis
            if np.dot(current_z, new_z) < 0:
                perp_vector = np.array([1, 0, 0]) if abs(current_z[0]) < 1e-6 else np.array([0, 1, 0])
                axis = np.cross(current_z, perp_vector)
                axis /= np.linalg.norm(axis)
                angle = np.pi
            else:
                angle = 0
                axis = np.array([1, 0, 0])  # Dummy, won't be used
        else:
            axis = cross_prod / axis_norm
            angle = np.arccos(np.clip(np.dot(current_z, new_z), -1.0, 1.0))

        # Create the alignment rotation
        align_rot = R.from_rotvec(angle * axis)

        # Apply to original rotation (compose: align_rot * rot)
        new_rot = align_rot * rot

    # Return new position and quaternion
    transform = np.eye(4)
    transform[:3, :3] = new_rot.as_matrix()
    transform[:3, 3] = new_pos
    return transform


def tube_info(scaffold_structure, valid_tube_index):
    tube_cylinder = []
    tube_mesh = []
    tube_pose = []
    tube_frames = []
    for tube_id, tube in scaffold_structure.items():
        if tube_id not in valid_tube_index:
            continue

        start_p = tube["from_p"]
        start_p[0] -= 0.2
        end_p = tube["to_p"]
        end_p[0] -= 0.2

        # get rotation quaternion from start_p to end_p
        direction = np.array(end_p) - np.array(start_p)
        direction = direction / np.linalg.norm(direction)
        # Assuming the cylinder's default orientation is along the z-axis
        z_axis = np.array([0, 0, 1])
        # Compute euler angles
        if np.allclose(direction, z_axis):
            roll, pitch, yaw = 0, 0, 0
        elif np.allclose(direction, -z_axis):
            roll, pitch, yaw = 0, np.pi, 0
        else:
            roll = np.arctan2(direction[0], direction[1])
            yaw = np.arccos(direction[2])
            pitch = 0
        qx, qy, qz, qw = euler_to_quaternion(roll, pitch, yaw)

        cylinder = Cylinder(
            name=f"tube{tube_id}",
            radius=0.024,
            height=np.linalg.norm(np.array(end_p) - np.array(start_p)),
            # pose is [x, y, z, qx, qy, qz, qw]
            pose=[(start_p[0] + end_p[0]) / 2, (start_p[1] + end_p[1]) / 2, (start_p[2] + end_p[2]) / 2, qx, qy, qz,
                  qw],
            color=[0.7, 0.7, 0.7, 1.0],
        )
        tube_cylinder.append(cylinder)

        # mesh = trimesh.creation.cylinder(radius=0.024, height=np.linalg.norm(np.array(end_p)-np.array(start_p)), sections=16)
        # mesh.apply_translation([(start_p[0]+end_p[0])/2, (start_p[1]+end_p[1])/2, (start_p[2]+end_p[2])/2])
        mesh = WorldConfig(cylinder=[cylinder]).get_mesh_world(merge_meshes=True).mesh[0]  # .get_trimesh_mesh()
        tube_mesh.append(mesh)

        tube_pose_rotate = []
        tube_pose_rotate_pos = []
        tube_pose_rotate_quat = []
        for deg in range(0, 360, 5):
            # # get rotation matrix
            # rotation_matrix = R.from_euler('xyz', [roll, pitch, yaw]).as_matrix()
            # # # Rotation along direction's normal by deg degrees
            # rotation_matrix = R.from_rotvec(direction * np.deg2rad(deg)).as_matrix() @ rotation_matrix
            # r = R.from_matrix(rotation_matrix)
            # qx, qy, qz, qw = r.as_quat()
            [new_roll, new_pitch, new_yaw] = rotate_euler((roll, pitch, yaw), direction, np.deg2rad(deg))
            qx, qy, qz, qw = euler_to_quaternion(new_roll, new_pitch, new_yaw)

            tube_pose_rotate_pos.append(
                [(start_p[0] + end_p[0]) / 2, (start_p[1] + end_p[1]) / 2, (start_p[2] + end_p[2]) / 2])
            tube_pose_rotate_quat.append([qx, qy, qz, qw])

            tube_pose_rotate = Pose(
                position=torch.tensor(tube_pose_rotate_pos, **tensor_args.as_torch_dict()),
                quaternion=torch.tensor(tube_pose_rotate_quat, **tensor_args.as_torch_dict())
            )

        tube_pose.append(tube_pose_rotate)
    return tube_cylinder, tube_mesh, tube_pose


def tube_frame(scaffold_structure, valid_tube_index):
    frames = []
    for tube_id, tube in scaffold_structure.items():
        if tube_id not in valid_tube_index:
            continue

        start_p = np.array(tube["from_p"])
        start_p[0] -= 0.2
        end_p = np.array(tube["to_p"])
        end_p[0] -= 0.2

        new_pos = 0.5 * (end_p + start_p)
        frame= update_frame([0, 0, 0, 1], new_pos, start_p, end_p)
        frames.append(frame)
    return frames


def generate_rotated_frames(original_transform, angles_degrees=None, num_steps=72):
    """
    Generates a list of new 4x4 transformation matrices by rotating the original transform around its local z-axis.

    Parameters:
    - original_transform: np.array (4,4) - The original 4x4 transformation matrix.
    - angles_degrees: list or np.array - Specific angles in degrees to rotate by. If None, uses linspace from 0 to 360.
    - num_steps: int - Number of steps if angles_degrees is None (includes 0, excludes 360 if full circle).

    Returns:
    - rotated_transforms: list of np.array (4,4) - List of rotated transformation matrices.
    """
    if angles_degrees is None:
        angles_degrees = np.linspace(0, 360, num_steps + 1)[:-1]  # 0 to almost 360, avoiding duplicate

    R_original = original_transform[:3, :3]
    t = original_transform[:3, 3]

    rotated_transforms = []
    pos = []
    rot = []
    for theta_deg in angles_degrees:
        theta_rad = np.deg2rad(theta_deg)
        # Rotation matrix around z-axis
        R_z = np.array([
            [np.cos(theta_rad), -np.sin(theta_rad), 0],
            [np.sin(theta_rad), np.cos(theta_rad), 0],
            [0, 0, 1]
        ])
        # Local rotation: post-multiply (right-multiply) the z-rotation
        new_R = R_original @ R_z
        new_transform = np.eye(4)
        new_transform[:3, :3] = new_R
        new_transform[:3, 3] = t
        rotated_transforms.append(new_transform[np.newaxis, :, :])
        rot.append(new_R[np.newaxis, :, :])
        # quat.append(R.from_matrix(new_R).as_quat()[np.newaxis, :, :])
        pos.append(t[np.newaxis, :])

    rot = torch.tensor(np.vstack(rot), **tensor_args.as_torch_dict())
    pos = torch.tensor(np.vstack(pos), **tensor_args.as_torch_dict())

    poses = Pose(position=pos, rotation=rot)
    return np.vstack(rotated_transforms), poses

In [5]:
import numpy as np
from typing import List, Dict
import xml.etree.ElementTree as ET


def get_geom_bounds(geom: Dict) -> Dict:
    """Compute world bounds for a geom based on type, pos, euler."""
    pos = geom['pos']
    size = geom['size']
    euler = geom['euler']

    # Rotation matrix from euler (xyz order)
    rx = np.array([[1, 0, 0], [0, np.cos(euler[0]), -np.sin(euler[0])], [0, np.sin(euler[0]), np.cos(euler[0])]])
    ry = np.array([[np.cos(euler[1]), 0, np.sin(euler[1])], [0, 1, 0], [-np.sin(euler[1]), 0, np.cos(euler[1])]])
    rz = np.array([[np.cos(euler[2]), -np.sin(euler[2]), 0], [np.sin(euler[2]), np.cos(euler[2]), 0], [0, 0, 1]])
    R = rz @ ry @ rx

    if geom['type'] == 'box':
        half_sizes = size
        corners_local = np.array(np.meshgrid(*[[-1, 1]] * 3, indexing='ij')).T.reshape(-1, 3) * half_sizes
        corners_world = (R @ corners_local.T).T + pos
        min_bounds = np.min(corners_world, axis=0)
        max_bounds = np.max(corners_world, axis=0)
    elif geom['type'] == 'cylinder':
        # Approximate bounds: radius in perpendicular planes, half-length along axis
        axis = R @ np.array([0, 0, 1])  # Axis direction
        perp1 = R @ np.array([1, 0, 0])
        perp2 = R @ np.array([0, 1, 0])
        half_length = size[1]
        radius = size[0]
        # Bounds along axis +/- half_length, perp +/- radius
        min_bounds = pos - half_length * np.abs(axis) - radius * (np.abs(perp1) + np.abs(perp2))
        max_bounds = pos + half_length * np.abs(axis) + radius * (np.abs(perp1) + np.abs(perp2))
    else:
        raise ValueError(f"Unsupported geom type: {geom['type']}")

    return {'min': min_bounds, 'max': max_bounds}


def compute_facade_contour(facade_geoms, depth=0.6, gap=0.2):
    """Compute segmented contour for scaffold: y segments with corresponding min_x for front."""
    y_bounds = set()
    for geom in facade_geoms:
        bounds = get_geom_bounds(geom)
        y_bounds.add(bounds['min'][1])
        y_bounds.add(bounds['max'][1])

    y_sorted = sorted(y_bounds)
    segments = []
    nx = []
    for i in range(len(y_sorted) - 1):
        y_min = y_sorted[i]
        y_max = y_sorted[i + 1]
        mid_y = (y_min + y_max) / 2
        active_min_x = np.inf
        for geom in facade_geoms:
            bounds = get_geom_bounds(geom)
            if bounds['min'][1] <= mid_y <= bounds['max'][1]:
                active_min_x = min(active_min_x, bounds['min'][0])
        if active_min_x < np.inf:
            segments.append({'y_min': y_min, 'y_max': y_max, 'min_x': active_min_x})
            nx.append(active_min_x)

    for seg in segments:
        min_x = seg['min_x']
        if min_x < max(nx):
            seg['min_x'] = max(nx) - depth - gap
        else:
            seg['min_x'] -= gap
    return segments


def plan_scaffold(facade_geoms: List[Dict], contour_segments: List[Dict], depth: float = 0.6, layer_height: float = 0.5, gap: float = 0.2) -> Dict:
    """Plan scaffold structure: sections, layers based on height."""
    # Max height from geoms
    max_height = max(get_geom_bounds(g)['max'][2] for g in facade_geoms)
    num_layers = int(np.ceil(max_height / layer_height))
    horizontal_levels = np.linspace(0, max_height, num_layers + 1)
    vertical_half_lengths = [layer_height / 2] * num_layers
    z_centers = (horizontal_levels[:-1] + horizontal_levels[1:]) / 2


    # ---- 重排 contour_segments ----
    # 依 min_x 由大到小排序；Python sort 是 stable，
    # 所以同一個 min_x 之間會保持原本的順序。

    contour_segments_sorted = sorted(
        contour_segments,
        key=lambda seg: seg["min_x"],
        reverse=True
    )


    sections = []
    for seg in contour_segments_sorted:
        sections.append({
            'y_start': seg['y_min'],
            'y_end': seg['y_max'],
            'x_front': seg['min_x'] - depth,
            'x_back': seg['min_x'],
            'add_diagonal': (seg['y_max'] - seg['y_min'] > 0.5), # only give true and false
        })

    return {
        'sections': sections,
        'num_layers': num_layers,                       # order of the layers
        'horizontal_levels': horizontal_levels,         # order of the layers
        'vertical_half_lengths': vertical_half_lengths, # order of the layers
        'z_centers': z_centers                          # order of the layers
    }


def generate_scaffold_xml(facade_xml_str: str,
                          gap: float = 0.2,
                          depth: float = 0.6,
                          layer_height: float = 0.8,
                          ):
    root = ET.fromstring(facade_xml_str)
    facade_geoms = []
    tube_info = {}
    for geom in root.findall('.//geom'):
        geom_dict = geom.attrib
        geom_dict['size'] = np.fromstring(geom_dict.get('size', '0 0 0'), sep=' ')
        geom_dict['pos'] = np.fromstring(geom_dict.get('pos', '0 0 0'), sep=' ')
        geom_dict['euler'] = np.fromstring(geom_dict.get('euler', '0 0 0'), sep=' ')
        facade_geoms.append(geom_dict)

    # --- 1) 原本的輪廓 & scaffold 計畫 ---
    contour_segments = compute_facade_contour(facade_geoms)
    plan = plan_scaffold(facade_geoms, contour_segments, depth, layer_height)

    # --- 2) 在這裡插入 GridPathPlanner：取得 sequence 並印出 ---
    # planner = GridPathPlanner(1.0, contour_segments, plan["num_layers"])
    # pos_sequences = planner.build_sequences_pos
    # 例如印第一個 segment 的所有 from/to
    # for i, (p_from, p_to) in enumerate(pos_sequences[0]):
    #     print(f"step {i}: from {p_from} to {p_to}")


    # planner.print_info()  # 這裡會印出 sorted segments 和每個 (layer, seg) 的 steps

    tube_radius = 0.024
    rgba = "0.7 0.7 0.7 1"
    material = "metal_mat"

    xml = '<?xml version="1.0" ?>\n<mujoco>\n'
    xml += '    <compiler angle = "radian"/>\n'
    xml += '    <option gravity="0 0 -9.81" timestep="0.002"/>\n'
    xml += '    <asset>\n'  # Assume assets are same
    xml += '        <texture name="wood" type="2d" builtin="flat" rgb1="0.8 0.6 0.4" rgb2="0.6 0.4 0.2" width="100" height="100"/>\n'
    xml += '        <material name="wood_mat" texture="wood" specular="0.5" shininess="0.5"/>\n'
    xml += '        <texture name="metal" type="2d" builtin="flat" rgb1="0.7 0.7 0.7" rgb2="0.5 0.5 0.5" width="100" height="100"/>\n'
    xml += '        <material name="metal_mat" texture="metal" specular="0.8" shininess="0.8"/>\n'
    xml += '        <texture name="groundplane" type="2d" builtin="checker" rgb1="0.2 0.3 0.4" rgb2="0.1 0.2 0.3" width="300" height="300"/>\n'
    xml += '        <material name="groundplane" texture="groundplane" texuniform="true" texrepeat="5 5" reflectance="0.2"/>\n'
    xml += '    </asset>\n'
    xml += '    <worldbody>\n'
    xml += '        <light pos="1 0 3.5" dir="0 0 -1" directional="true"/>\n'
    xml += '        <geom name="floor" size="0 0 0.05" type="plane" material="groundplane"/>\n'

    # Add facade geoms
    for geom in facade_geoms:
        pos_str = ' '.join(f"{p:.1f}" for p in geom['pos'])
        size_str = ' '.join(f"{s:.3f}" for s in geom['size'])
        euler_str = ' '.join(f"{e:.2f}" for e in geom['euler'])
        rgba_str = geom.get('rgba', '0.8 0.6 0.4 1')
        mat_str = geom.get('material', 'wood_mat')
        xml += f'        <geom name="{geom["name"]}" type="{geom["type"]}" size="{size_str}" pos="{pos_str}" euler="{euler_str}" rgba="{rgba_str}" material="{mat_str}"/>\n'

    tube_id = 0
    for sec in plan['sections']:
        y_start = sec['y_start']
        y_end = sec['y_end']
        x_front = sec['x_front']
        x_back = sec['x_back']
        add_diagonal = sec['add_diagonal']
        offset_0_025 =  0.025
        offset_0_05 =  0.05
        # Vertical poles
        for layer in range(plan['num_layers']):
            half_length = plan['vertical_half_lengths'][layer]
            z_center = plan['z_centers'][layer]
            # 0
            from_p = [x_back, y_start, z_center - half_length +offset_0_025]
            to_p = [x_back, y_start, z_center + half_length -offset_0_025]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.3f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 1
            from_p = [x_back, y_end, z_center - half_length+offset_0_025]
            to_p = [x_back, y_end, z_center + half_length -offset_0_025]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.3f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 2
            from_p = [x_back, y_end -offset_0_05 , z_center + half_length -offset_0_05 ]
            to_p = [x_back, y_start +offset_0_05 ,z_center - half_length  +offset_0_05 ]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 3
            from_p = [x_back, y_end -offset_0_025 , z_center + half_length ]
            to_p = [x_back, y_start +offset_0_025 ,z_center + half_length ]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 4
            from_p = [x_back-offset_0_025, y_end , z_center + half_length ]
            to_p = [x_front+offset_0_025, y_end ,z_center + half_length ]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 5
            from_p = [x_back-offset_0_025, y_start , z_center + half_length ]
            to_p = [x_front+offset_0_025, y_start ,z_center + half_length ]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 6
            from_p = [x_front, y_end , z_center - half_length +offset_0_025]
            to_p = [x_front, y_end ,z_center + half_length  -offset_0_025]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 7
            from_p = [x_front, y_start , z_center - half_length +offset_0_025]
            to_p = [x_front, y_start ,z_center + half_length  -offset_0_025]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 8
            from_p = [x_front, y_end -offset_0_05 , z_center - half_length +offset_0_05 ]
            to_p = [x_front, y_start +offset_0_05 ,z_center + half_length  -offset_0_05 ]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

            # 9
            from_p = [x_front, y_end -offset_0_025 , z_center + half_length ]
            to_p = [x_front, y_start +offset_0_025 ,z_center + half_length ]
            xml += f'        <geom name="tube{tube_id}" type="cylinder" fromto="{from_p[0]:.3f} {from_p[1]:.3f} {from_p[2]:.3f} {to_p[0]:.3f} {to_p[1]:.3f} {to_p[2]:.2f}" size="{tube_radius}" rgba="{rgba}" material="{material}"/>\n'
            tube_info[tube_id] = {"from_p": from_p, "to_p": to_p}
            tube_id += 1

    # key: (x1,y1,z1,x2,y2,z2)（端點排序後、四捨五入），value: [tube_id1, tube_id2, ...]
    segment_dict = {}

    for tid, seg in tube_info.items():
        p1 = np.array(seg["from_p"], dtype=float)
        p2 = np.array(seg["to_p"], dtype=float)

        # 忽略方向：把端點按座標字典序排序，A→B 和 B→A 都變成同一個 key
        if tuple(p1) <= tuple(p2):
            a, b = p1, p2
        else:
            a, b = p2, p1

        # 四捨五入，避免 0.3000000004 之類的浮點數誤差
        key = tuple(np.round(np.concatenate([a, b]), 3))

        if key not in segment_dict:
            segment_dict[key] = []
        segment_dict[key].append(tid)

    overlapped_tube_ids = []
    valid_tube_ids_set = set()

    for key, ids in segment_dict.items():
        ids_sorted = sorted(ids)
        # 只要這個幾何 segment 出現超過一次，就視為重疊群組
        if len(ids_sorted) > 1:
            overlapped_tube_ids.extend(ids_sorted)
        # 無論有沒有重疊，這組裡「最小的 id」保留
        valid_tube_ids_set.add(ids_sorted[0])

    # 排序一下方便看
    overlapped_tube_ids = sorted(set(overlapped_tube_ids))
    valid_tube_index = sorted(valid_tube_ids_set)

    # 實際被丟掉的那幾根（重疊群組裡除了第一根以外）
    removed_tube_ids = sorted(set(overlapped_tube_ids) - valid_tube_ids_set)

    # print("Overlapped tube indices (all in overlapping groups):", overlapped_tube_ids)
    # print("Removed overlapped tube indices (discarded):", removed_tube_ids)
    print("Valid tube indices (kept):", valid_tube_index)


    xml += '    </worldbody>\n</mujoco>\n'
    return xml, tube_info, valid_tube_index


def get_design():
    # get different designs by setting different wall, col1, and col2 locations
    facade_xml = '''
    <worldbody>
        <geom name="wall" type="box" size="2.0 0.025 2.4" pos="3.2 0 0" euler="0 0 1.57" rgba="0.8 0.6 0.4 1" material="wood_mat"/>
        <geom name="col1" type="cylinder" size="0.4 1.2" pos="3.2 -0.8 1.2" euler="0 0 1.57" rgba="0.8 0.6 0.4 1" material="wood_mat"/>
        <geom name="col2" type="cylinder" size="0.4 1.2" pos="3.2 0.8 1.2" euler="0 0 1.57" rgba="0.8 0.6 0.4 1" material="wood_mat"/>
    </worldbody>
    '''

    # scaffold_structure[id] = {"start_pos":[], "end_pos": []}
    xml, scaffold_structure, valid_tube_index = generate_scaffold_xml(
        facade_xml_str=facade_xml,
    )

    return xml, scaffold_structure, valid_tube_index





In [8]:
# torch
import torch
import numpy as np
import copy
import trimesh
# cuRobo
from curobo.cuda_robot_model.cuda_robot_model import CudaRobotModel, CudaRobotModelConfig
from curobo.types.base import TensorDeviceType
from curobo.types.robot import RobotConfig
from curobo.util_file import get_robot_path, join_path, load_yaml
from robofab.mqtt import publish_robot_trajs, publish_clear, publish_world, publish_frames, publish_grasp_object
from robofab.place import poses_to_frames
from curobo.geom.types import WorldConfig, Mesh, Cylinder, Cuboid
from curobo.geom.types import Pose


# convenience function to store tensor type and device
tensor_args = TensorDeviceType()

place_world = WorldConfig(
    cylinder=[
        Cylinder(
            name="col1",
            radius=0.4,
            height=1.2 * 2,
            pose=[3.2, -0.8, 1.2, np.sin(1.57 / 2), 0, 0, np.cos(1.57 / 2)],
            color=[0.8, 0.6, 0.4, 1.0],
        ),
        Cylinder(
            name="col2",
            radius=0.4,
            height=1.2 * 2,
            pose=[3.2, 0.8, 1.2, np.sin(1.57 / 2), 0, 0, np.cos(1.57 / 2)],
            color=[0.8, 0.6, 0.4, 1.0],
        ),
        # tube_cylinder[2]
    ],
    cuboid=[Cuboid(
        name="wall",
        dims=[2.0 * 2, 0.025 * 2, 2.4],
        pose=[3.2, 0, 1.2, np.sin(1.57 / 2), 0, 0, np.cos(1.57 / 2)],
        color=[0.8, 0.6, 0.4, 1.0],
    )]
)


robot_config_file, robot_kin_model = load_kuka_kin_model(0.15)
retract_q = robot_kin_model.retract_config.cpu().numpy().reshape(1, -1)
publish_clear()
publish_robot_trajs(retract_q, 0.15)
xml, scaffold_structure, valid_tube_index = get_design()
fully_placed_world, tube_mesh, tube_cylinder = gen_fully_placed_world(copy.deepcopy(scaffold_structure), valid_tube_index, place_world)

# visualize
publish_clear()
publish_robot_trajs(retract_q, 0.15)
publish_world(fully_placed_world, merged=True)

tube_frames = tube_frame(scaffold_structure, valid_tube_index)
init_frames = [i[np.newaxis, :, :] for i in tube_frames]
init_frames = np.vstack(init_frames)
publish_frames(init_frames)

Valid tube indices (kept): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 99, 102, 103, 104, 105, 106, 107, 108, 109, 112, 113, 114, 115, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127, 128, 129, 132, 133, 134, 135, 136, 137, 138, 139, 142, 143, 144, 145, 146, 147, 148, 149]


In [7]:
tube_id = 0
rotated_frames, tube_poses = generate_rotated_frames(tube_frames[tube_id], num_steps=72)
publish_frames(rotated_frames)

# place_result = plan_ik(tube_poses, place_world, 0.15)
# traj, attached_object_for_rendering = compute_pick_place(tube_id, tube_poses[tube_id], tube_mesh[tube_id], place_world, robot_kin_model)
# publish_robot_trajs(traj, 0.15)
# publish_grasp_object(attached_object_for_rendering)

In [ ]:
final_trajs = np.empty((0, len(robot_kin_model.joint_names)))
save_data = {
    'seq': [],
    'traj': {},
}
for tube_id in range(len(tube_mesh)):
    try:
        rotated_frames, tube_poses = generate_rotated_frames(tube_frames[tube_id], num_steps=72)
        publish_frames(rotated_frames)

        traj, attached_object_for_rendering = compute_pick_place(tube_id, tube_poses, tube_mesh[tube_id], place_world, robot_kin_model)
        final_trajs = np.vstack([final_trajs, traj])
        # update place world
        place_world = WorldConfig(
            cylinder=[*place_world.cylinder, tube_cylinder[tube_id]],
            cuboid=[*place_world.cuboid],
        )
        publish_robot_trajs(final_trajs, 0.15)
        publish_grasp_object(attached_object_for_rendering)
        publish_world(place_world, merged=True)

        cur_traj = {
            'id': tube_id,
            'tube_traj': traj,
            'attached_object_for_rendering': attached_object_for_rendering
        }

        save_data['traj'][tube_id] = cur_traj
        save_data['seq'].append(tube_id)

        print(f"Successfully processed tube_id {tube_id}")
    except RuntimeError as e:
        print(f"Error processing tube_id {tube_id}: {e}")

# import pickle
# with open("save_data.pkl", "wb") as f:
#     pickle.dump(save_data, f)

# tube_cylinder, tube_mesh, tube_pose = tube_info(copy.deepcopy(scaffold_structure), valid_tube_index)
# for tube_p in tube_pose:
#     frame = poses_to_frames(tube_p)

# tube_id = 15  # 21
# frames = poses_to_frames(tube_pose[tube_id])
# publish_frames(init_frames)

Error processing tube_id 0: No solution found!
Successfully processed tube_id 1
Successfully processed tube_id 2
Successfully processed tube_id 3
Successfully processed tube_id 4


only got 48/50 samples!


Successfully processed tube_id 5
Successfully processed tube_id 6
Successfully processed tube_id 7
Successfully processed tube_id 8
Successfully processed tube_id 9
Error processing tube_id 10: No solution found!
Successfully processed tube_id 11
Successfully processed tube_id 12
Successfully processed tube_id 13
Successfully processed tube_id 14
Error processing tube_id 15: No solution found!


only got 49/50 samples!


Error processing tube_id 16: No solution found!
Successfully processed tube_id 17
Successfully processed tube_id 18
Successfully processed tube_id 19
Error processing tube_id 20: No solution found!
Successfully processed tube_id 21
Error processing tube_id 22: No solution found!
Error processing tube_id 23: No solution found!
Error processing tube_id 24: No solution found!
Error processing tube_id 25: No solution found!
Successfully processed tube_id 26
Successfully processed tube_id 27


...

Successfully processed tube_id 75
Successfully processed tube_id 76


only got 45/50 samples!


Successfully processed tube_id 77
Successfully processed tube_id 78
Successfully processed tube_id 79
Successfully processed tube_id 80
Error processing tube_id 81: No solution found!
Error processing tube_id 82: No solution found!
Error processing tube_id 83: No solution found!
Error processing tube_id 84: No solution found!
Error processing tube_id 85: No solution found!


only got 49/50 samples!


Successfully processed tube_id 86
Successfully processed tube_id 87


Valid tube indices (kept): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 99, 102, 103, 104, 105, 106, 107, 108, 109, 112, 113, 114, 115, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127, 128, 129, 132, 133, 134, 135, 136, 137, 138, 139, 142, 143, 144, 145, 146, 147, 148, 149]


In [ ]:
import pickle

with open("save_data.pkl", "rb") as f:
    save_data = pickle.load(f)


visual_world = WorldConfig(
    cylinder=[
        Cylinder(
            name="col1",
            radius=0.4,
            height=1.2 * 2,
            pose=[3.2, -0.8, 1.2, np.sin(1.57 / 2), 0, 0, np.cos(1.57 / 2)],
            color=[0.8, 0.6, 0.4, 1.0],
        ),
        Cylinder(
            name="col2",
            radius=0.4,
            height=1.2 * 2,
            pose=[3.2, 0.8, 1.2, np.sin(1.57 / 2), 0, 0, np.cos(1.57 / 2)],
            color=[0.8, 0.6, 0.4, 1.0],
        ),
        # tube_cylinder[2]
    ],
    cuboid=[Cuboid(
        name="wall",
        dims=[2.0 * 2, 0.025 * 2, 2.4],
        pose=[3.2, 0, 1.2, np.sin(1.57 / 2), 0, 0, np.cos(1.57 / 2)],
        color=[0.8, 0.6, 0.4, 1.0],
    )]
)

for tube_id in save_data['seq']:
    traj = save_data['traj'][tube_id]['tube_traj']
    attached_object_for_rendering = save_data['traj'][tube_id]['attached_object_for_rendering']
    visual_world = WorldConfig(
    cylinder=[*visual_world.cylinder, tube_cylinder[tube_id]],
    cuboid=[*visual_world.cuboid],
    )
    publish_robot_trajs(traj, 0.15)
    publish_grasp_object(attached_object_for_rendering)
    publish_world(place_world, merged=True)

